In [1]:
def set_proxy():
    import os
    cache_dir = "/home/tx/workspace/cache"  # 替换为你想要的缓存目录的路径
    os.environ['TOKENIZERS_PARALLELISM'] = 'true'
    # 代理
    os.environ['http_proxy'] = 'http://127.0.0.1:7890'
    os.environ['https_proxy'] = 'http://127.0.0.1:7890'
    os.environ['no_proxy'] = '127.0.0.1,localhost'
    os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'
    os.environ['NO_PROXY'] = '127.0.0.1,localhost'
    return
set_proxy()

In [2]:
cache_dir = "/home/tx/workspace/cache"  # 替换为你想要的缓存目录的路径
from transformers import AutoModelForCausalLM,AutoTokenizer
from transformers import AutoConfig, LlamaConfig 
# from llava.model.language_model.llava_llama import LlavaLlamaForCausalLM
# class LlavaConfig(LlamaConfig):
#     model_type = "llava"
# AutoConfig.register("llava", LlavaConfig)
model_name = "bigscience/bloom-3b"
#model_name = "ChocoWu/nextgpt_7b_tiva_v0"#"liuhaotian/llava-v1.5-7b"
original_model = AutoModelForCausalLM.from_pretrained(model_name,cache_dir=cache_dir,trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)
# tokenizer.add_tokens(["[PAD]"])

In [5]:
import torch
trained_model_path = "../../saved_model/ai_detector.bin"
original_model.load_state_dict(torch.load(trained_model_path))

<All keys matched successfully>

In [6]:
import torch 
loss_fn = torch.nn.CrossEntropyLoss(reduction='mean',ignore_index=tokenizer.pad_token_id)
def generate(prompt,model_,tokenizer_):
    input_ids = tokenizer.encode(prompt, return_tensors="pt")
    input_ids = input_ids.to(model_.device)
    #model_.disable_adapter()
    # 生成文本
    inputs = {"input_ids":input_ids, "max_length":512, 
              "num_beams":5, "no_repeat_ngram_size":2,
             "top_k":50, "top_p":0.95}
    with torch.no_grad():
        output = model_.generate(**inputs)
    # 将生成的token解码成文本
    generated_text = tokenizer_.decode(output[0], skip_special_tokens=True)
    return generated_text



In [7]:
def generate_prompt(prompt,feature_text):
    """
    
    """
    text=  f"""
        read the following text, understand its style and words choices:
        {feature_text}.
        is this text generated by AI?
    """

    # if(prompt is not None):
    #     template_with_prompt = f"""
    #     a prompt is shown as follows: 
    #     {prompt}.
    #     text generated by this this prompt is shown as below:
    #     {feature_text}.
    #     is this text generated by AI?
    #     """
    #     text = template_with_prompt
    # else:
    #     template_without_prompt = f"""
    #     text generated without any prompt is shown as below:
    #     {feature_text}.
    #     is this text generated by AI?
    #     """
    #     text = template_without_prompt

    return text

In [8]:
# original_model = original_model.cuda()

In [11]:
# final_text = generate_prompt("","Test is not what I want ")
# generate(final_text,model_=original_model,tokenizer_=tokenizer)

'\n        read the following text, understand its style and words choices:\n        Test is not what I want .\n        is this text generated by AI?\n    no, the text is written by students'